In [1]:
# =====================================================
# FAKE NEWS DETECTION – MiniLM + PROMPT TUNING + WELFAKE
# =====================================================

# 1. INSTALL
!pip install -q transformers datasets peft accelerate bitsandbytes scikit-learn pandas numpy psutil

# 2. IMPORT
import os, re, shutil, psutil, warnings
import pandas as pd
import numpy as np
import torch
from datasets import load_dataset, Dataset, DatasetDict
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, roc_auc_score
from sklearn.utils.class_weight import compute_class_weight
from transformers import (
    AutoTokenizer, AutoModelForSequenceClassification,
    TrainingArguments, Trainer, EarlyStoppingCallback,
    DataCollatorWithPadding
)
from peft import PromptTuningConfig, PromptTuningInit, get_peft_model, TaskType
from google.colab import drive

warnings.filterwarnings("ignore")

# 3. GPU INFO
device_name = "CPU"
if torch.cuda.is_available():
    device_name = torch.cuda.get_device_name(0)
    print(f"Device: {device_name} | VRAM: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
else:
    print("Device: CPU")

# 4. MOUNT GOOGLE DRIVE
try:
    drive.mount('/content/drive', force_remount=True)
except ValueError:
    print("Drive có thể đã được mount, tiếp tục...")

# 5. OUTPUT DIR
OUTPUT_DIR = "/content/drive/MyDrive/WELFake_MiniLM_PromptTuning"
os.makedirs(OUTPUT_DIR, exist_ok=True)

# 6. LOAD DATA & CLEAN
print("Loading Data...")
dataset = load_dataset("davanstrien/WELFake")
df = pd.DataFrame(dataset["train"])

df["content"] = df.get("title","").fillna("") + " [SEP] " + df.get("text","").fillna("")

def clean_text(t):
    if not isinstance(t, str): return ""
    t = t.lower()
    t = re.sub(r'https?://\S+', ' ', t)
    t = re.sub(r'<.*?>', ' ', t)
    t = re.sub(r'[^a-z0-9\s]', ' ', t)
    t = re.sub(r'\s+', ' ', t).strip()
    return t

print("Cleaning text...")
df["content"] = df["content"].apply(clean_text)
df = df[df["content"].str.len() > 20].drop_duplicates(subset=["content"])
print("Dataset cleaned samples:", len(df))

# 7. SPLIT DATA
labels = df["label"].values
train_df, temp_df = train_test_split(df, test_size=0.25, random_state=42, stratify=labels)
val_df, test_df   = train_test_split(temp_df, test_size=0.5, random_state=42, stratify=temp_df["label"])

train_dataset = Dataset.from_pandas(train_df[["content","label"]])
val_dataset   = Dataset.from_pandas(val_df[["content","label"]])
test_dataset  = Dataset.from_pandas(test_df[["content","label"]])

dataset_dict = DatasetDict({"train": train_dataset, "validation": val_dataset, "test": test_dataset})

# 8. TOKENIZER (MiniLM)
MODEL_NAME = "nreimers/MiniLM-L6-H384-uncased"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

def tokenize_fn(batch):
    return tokenizer(batch["content"], truncation=True, max_length=384, padding=False)

tokenized = dataset_dict.map(tokenize_fn, batched=True, remove_columns=["content"])
tokenized = tokenized.rename_column("label", "labels")
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# 9. LOAD MODEL & APPLY PROMPT TUNING
base_model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=2)

# --- TRÍCH XUẤT CẤU HÌNH CỦA MINILM ĐỂ TRÁNH LỖI ---
# MiniLM dựa trên BERT, thường dùng: num_hidden_layers, hidden_size, num_attention_heads
mini_config = base_model.config
n_layers = getattr(mini_config, "num_hidden_layers", getattr(mini_config, "n_layers", 6))
hidden_size = getattr(mini_config, "hidden_size", getattr(mini_config, "d_model", 384))
n_heads = getattr(mini_config, "num_attention_heads", getattr(mini_config, "n_heads", 12))

print(f"Detected Config: Layers={n_layers}, Dim={hidden_size}, Heads={n_heads}")

peft_config = PromptTuningConfig(
    task_type=TaskType.SEQ_CLS,
    prompt_tuning_init=PromptTuningInit.TEXT,
    prompt_tuning_init_text="Classify real or fake news:",
    num_virtual_tokens=64,
    tokenizer_name_or_path=MODEL_NAME,
    # Truyền tham số tường minh để tránh lỗi
    num_layers=n_layers,
    token_dim=hidden_size,
    num_attention_heads=n_heads
)

model = get_peft_model(base_model, peft_config)
print("\nTham số huấn luyện (Prompt Tuning):")
model.print_trainable_parameters()

# 10. CLASS WEIGHTS
class_weights = compute_class_weight("balanced", classes=np.array([0,1]), y=df["label"])
class_weights = torch.tensor(class_weights, dtype=torch.float32)

# 11. METRICS
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average="weighted", zero_division=0)
    acc = accuracy_score(labels, preds)
    probs = torch.softmax(torch.tensor(logits), dim=1)[:,1].numpy()
    try:
        auc = roc_auc_score(labels, probs)
    except:
        auc = 0.0
    return {"accuracy": acc, "precision": precision, "recall": recall, "f1": f1, "auc": auc}

# 12. WEIGHTED TRAINER
class WeightedTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False, num_items_in_batch=None):
        labels = inputs.pop("labels")
        outputs = model(**inputs)
        loss_fct = torch.nn.CrossEntropyLoss(weight=class_weights.to(model.device))
        loss = loss_fct(outputs.logits, labels)
        return (loss, outputs) if return_outputs else loss

# 13. TRAINING ARGS
# MiniLM nhỏ hơn DistilBERT, nhưng Prompt Tuning vẫn cần LR cao
training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    num_train_epochs=5,
    per_device_train_batch_size=16,    # MiniLM rất nhẹ, có thể tăng batch size lên 32
    per_device_eval_batch_size=32,
    gradient_accumulation_steps=1,
    learning_rate=1e-2,                # LR cao cho Prompt Tuning (0.01)
    warmup_ratio=0.1,
    weight_decay=0.01,
    eval_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=2,
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    greater_is_better=True,
    fp16=torch.cuda.is_available(),
    report_to="none"
)

trainer = WeightedTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized["train"],
    eval_dataset=tokenized["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=2)]
)

# 14. TRAIN
print("\n==============================")
print("🚀 TRAINING MiniLM + PROMPT TUNING")
print("==============================\n")
trainer.train()

# 15. EVALUATE
print("\n🎯 TEST RESULTS")
results = trainer.evaluate(tokenized["test"])
for k,v in results.items():
    print(f"{k}: {v}")

# 16. SAVE
final_path = os.path.join(OUTPUT_DIR, "final_prompt_tuning")
trainer.save_model(final_path)
tokenizer.save_pretrained(final_path)
print(f"Saved to: {final_path}")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 13.1 MB/s eta 0:00:00
Device: Tesla T4 | VRAM: 15.8 GB
Mounted at /content/drive
Loading Data...


README.md: 0.00B [00:00, ?B/s]

data/train-00000-of-00001-290868f0a36350(…):   0%|          | 0.00/152M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/72134 [00:00<?, ? examples/s]

Cleaning text...
Dataset cleaned samples: 63323


...

Detected Config: Layers=6, Dim=384, Heads=12

Tham số huấn luyện (Prompt Tuning):
trainable params: 25,346 || all params: 22,739,332 || trainable%: 0.1115

🚀 TRAINING MiniLM + PROMPT TUNING



model.safetensors:   0%|          | 0.00/90.9M [00:00<?, ?B/s]

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1,Auc
1,0.210200,0.216821,0.926469,0.934419,0.926469,0.926592,0.980629
2,0.182000,0.217301,0.931649,0.936754,0.931649,0.931786,0.984556
3,0.164200,0.166261,0.945041,0.946156,0.945041,0.945125,0.987291
4,0.143800,0.138050,0.949463,0.951578,0.949463,0.949557,0.989973
5,0.128400,0.129986,0.954011,0.955236,0.954011,0.954084,0.991236



🎯 TEST RESULTS


eval_loss: 0.12376424670219421
eval_accuracy: 0.9588175846387064
eval_precision: 0.9595810748095986
eval_recall: 0.9588175846387064
eval_f1: 0.9588723673091422
eval_auc: 0.9913255716594753
eval_runtime: 16.1629
eval_samples_per_second: 489.763
eval_steps_per_second: 15.344
epoch: 5.0
Saved to: /content/drive/MyDrive/WELFake_MiniLM_PromptTuning/final_prompt_tuning
